In [1]:
from openff.qcsubmit.datasets import load_dataset, OptimizationDataset
from openforcefield.utils.toolkits import GLOBAL_TOOLKIT_REGISTRY, OpenEyeToolkitWrapper
import os
import re

GLOBAL_TOOLKIT_REGISTRY.deregister_toolkit(OpenEyeToolkitWrapper())
print(GLOBAL_TOOLKIT_REGISTRY)

ToolkitRegistry containing The RDKit, AmberTools, Built-in Toolkit


# Preperation

Each partner has has their chossen molecules ran through the openff-benchmark pipeline to produce a reference qcsubmit dataset. Here we combine all of the datasets to gether keeping the partner identifiers and filter some unwanted elements out that were missed by the benchmark infastructure into a seprate dataset.

In [3]:
# load all datasets and filter out unwanted elements into a new dataset
# filter B, Si and I
filtered_dataset = OptimizationDataset(dataset_name="OpenFF Industry Benchmark Filtered v1.0", description="The set of molecules filtered from the season 1 openff industry benchmark public dataset. These molecules were filtered due to missing forcefield coverage and an issue with density fitting in psi4 regarding iodine.")

master_dataset = OptimizationDataset(dataset_name="OpenFF Industry Benchmark Season 1 v1.0", dataset_tagline="The public molecules from the OpenFF Industry Benchmark.", description="The combination of all publicly chossen compound sets by industry partners from the OpenFF season 1 industry benchmark.")

filtered_elements = {"B", "Si", "I"}
for ref_dir in ["boehringer", "genentech", "janssen", "merck", "roche", "swope", "xtalpi"]:

    dataset = load_dataset(os.path.join(ref_dir, "refdata.json.bz2"))
    
    for index, entry in dataset.dataset.items():
        # split the index to get the molecule id 
        mol_id = re.search("[A-Z]{3}-[0-9]{5}", index).group()
        mol_id += "-00"
        # get the symbols to quick filter elements
        qcmol_elements = set(entry.initial_molecules[0].symbols)
        if qcmol_elements.intersection(filtered_elements) != set():
            # add to the filter dataset
            if mol_id in filtered_dataset.dataset:
                filtered_dataset.dataset[mol_id].initial_molecules.extend(entry.initial_molecules)
            else:
                filtered_dataset.dataset[mol_id] = entry
        else:
            # add to the master dataset
            if mol_id in master_dataset.dataset:
                master_dataset.dataset[mol_id].initial_molecules.extend(entry.initial_molecules)
            else:
                master_dataset.dataset[mol_id] = entry
                

/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C15H25N2O has a net charge of 1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C24H31N2O4 has a net charge of 1.0.
  UserWarning,
Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 26)

/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C37H50N7O4 has a net charge of 1.0.
  UserWarning,
Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 4)

/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253:

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 38)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 8)

/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C6H13N2O2 has a net charge of 1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C16H22N3O6S has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C26H29O4 has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253

/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C12ClH10N2O5S has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C20H21N2O5S has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C9H11N2O4S has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C11H9NO5 has a net charge of -2.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C6H5N2O2 has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253

/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C17H15N4O2 has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C9H9O3 has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C25F3H24N2O2 has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C24H24N3O2 has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule BrC8H6O4S has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: U

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 6)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 6)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 8)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 8)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Bonds with undefined stereochemistry are:
 - Bond 14 (atoms 14-15 of element (C-N)

Warning (not error because allow_undefined_stereo=True): Unabl

...

/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C8H12NO3 has a net charge of 1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C10H13N2O has a net charge of 1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C18F2H16N5O2 has a net charge of 1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C14ClH8N2O3S has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C15H20N7O has a net charge of 1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: Us

/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C16F4H8NO3S has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C11ClH13N3O3S has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C10H15N2O2S has a net charge of 1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C12H14NO4S has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C23H23N2O2S2 has a net charge of 1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.p

/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C7H6NO4S has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C12H14NO5S has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C16H11O7 has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C17H15N2O3S has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C18Cl2H20N3O3S has a net charge of 1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253

/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C15ClH14N4O2S2 has a net charge of 1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C14H11N6O has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C9H9N2O3 has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C15F3H10N2O3 has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C12H15N4O2S2 has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py

/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C21ClH22N2O3S has a net charge of 1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C10H6N5O2S2 has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C14H4N2O8 has a net charge of -2.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C19ClH12N2O2 has a net charge of -1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C22ClH25N3O has a net charge of 1.0.
  UserWarning,
/Users/joshua/miniconda3/envs/qcsubmit/lib/python3.7/site-packages/openff/qcsubmit/validators.py

In [4]:
filtered_dataset.n_molecules

17

In [5]:
filtered_dataset.n_records

118

In [6]:
filtered_dataset.dataset.keys()

dict_keys(['GNT-00628-00', 'GNT-00484-00', 'GNT-00454-00', 'GNT-00768-00', 'GNT-00660-00', 'MRK-01201-00', 'MRK-01551-00', 'MRK-00791-00', 'XTP-00744-00', 'XTP-00492-00', 'XTP-00340-00', 'XTP-00305-00', 'XTP-00728-00', 'XTP-00241-00', 'XTP-00538-00', 'XTP-01152-00', 'XTP-00958-00'])

In [7]:
from rdkit.Chem import Descriptors
import numpy as np

In [9]:
confs = np.array([len(mol.conformers) for mol in master_dataset.molecules])
elements = set()
for entry in master_dataset.dataset.values():
    elements.update(entry.initial_molecules[0].symbols)

print("Unique list of elements          ", elements)
print("Number of unique molecules       ", master_dataset.n_molecules)
print("Number of filtered molecules     ", filtered_dataset.n_molecules)
print("Number of optimizations         ", master_dataset.n_records)
print("Number of conformers min mean max",
      confs.min(), "{:6.2f}".format(confs.mean()), confs.max())

masses = []
for molecule in master_dataset.molecules:
    rd_mol = molecule.to_rdkit()
    mass = Descriptors.ExactMolWt(rd_mol)
    masses.append(mass)

print(f'Mean molecular weight: {np.mean(np.array(masses)):.2f}')
print(f'Max molecular weight: {np.max(np.array(masses)):.2f}')
print("Charges:", sorted(set(m.total_charge/m.total_charge.unit for m in master_dataset.molecules)))


Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 26)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 4)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 8)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 38)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 8)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RD

Unique list of elements           {'N', 'F', 'Cl', 'C', 'H', 'O', 'Br', 'P', 'S'}
Number of unique molecules        9068
Number of filtered molecules      17
Number of optimizations          69672
Number of conformers min mean max 1   7.65 10


Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 26)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 4)

RDKit WARNING: [13:50:18] Conflicting single bond directions around double bond at index 27.
RDKit WARNING: [13:50:18]   BondStereo set to STEREONONE and single bond directions set to NONE.
Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 8)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 38)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from 

Mean molecular weight: 341.97
Max molecular weight: 1104.40


Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 26)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 4)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 8)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 38)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Undefined chiral centers are:
 - Atom C (index 8)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RD

Charges: [-2.0, -1.0, 0.0, 1.0, 2.0]


In [11]:
from openff import qcsubmit
import openforcefield
import rdkit
from pprint import pprint

In [12]:
# change the metadata
master_dataset.metadata.submitter = "jthorton"
master_dataset.metadata.long_description_url = ("https://github.com/openforcefield/qca-dataset-submission/tree/master/"
                                                "submissions/"
                                                "2021-03-30-OpenFF-Industry-Benchmark-Season-1-v1.0")
master_dataset.metadata.elements = elements
master_dataset.provenance = {"openff-qcsubmit": qcsubmit.__version__,
                            "openff-toolkit": openforcefield.__version__,
                            "rdkit": rdkit.__version__,
                            "openff-benchamrk": "2021.03.17.0"}



In [13]:
pprint(master_dataset.metadata.dict())

{'collection_type': 'OptimizationDataset',
 'creation_date': datetime.date(2021, 3, 30),
 'dataset_name': 'OpenFF Industry Benchmark Season 1 v1.0',
 'elements': {'N', 'F', 'Cl', 'C', 'H', 'O', 'Br', 'P', 'S'},
 'long_description': 'The combination of all publicly chossen compound sets by '
                     'industry partners from the OpenFF season 1 industry '
                     'benchmark.',
 'long_description_url': HttpUrl('https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/2021-03-30-OpenFF-Industry-Benchmark-Season-1-v1.0', scheme='https', host='github.com', tld='com', host_type='domain', path='/openforcefield/qca-dataset-submission/tree/master/submissions/2021-03-30-OpenFF-Industry-Benchmark-Season-1-v1.0'),
 'short_description': 'The public molecules from the OpenFF Industry '
                      'Benchmark.',
 'submitter': 'jthorton'}


In [15]:
master_dataset.export_dataset("dataset.json.bz2")
master_dataset.molecules_to_file("dataset.smi", "smi")

# Dataset visualization 
A manual workaround for rdkit not correclty producing a pdf.

In [ ]:
from rdkit.Chem import AllChem, Draw
images = []
molecules = []
for data in master_dataset.dataset.values():
    rdkit_mol = data.get_off_molecule(include_conformers=False).to_rdkit()
    AllChem.Compute2DCoords(rdkit_mol)
    molecules.append(rdkit_mol)
# evey 24 molecules split the page
for i in range(0, len(molecules), 24):
    mol_chunk = molecules[i : i + 24]

    # now make the image
    image = Draw.MolsToGridImage(
        mol_chunk,
        molsPerRow=4,
        subImgSize=(500, 500),
        returnPNG=False
    )
    # write the pdf to bytes and pass straight to the pdf merger
    images.append(image)


In [ ]:
images[0].save("dataset.pdf", append_images=images[1:], save_all=True)

# Filter Dataset
The dataset of filtered molecules is also saved as filtered.json.bz2 so it is not submitted by qcsubmit.

In [16]:
filtered_dataset.export_dataset("filtered.json.bz2")